<a href="https://colab.research.google.com/github/UmutMD/monkey-classification-cnn-mobilenet/blob/main/10_Monkey_Species_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About Dataset
Content
The dataset consists of two files, training and validation. Each folder contains 10 subforders labeled as n0~n9, each corresponding a species form Wikipedia's monkey cladogram. Images are 400x300 px or larger and JPEG format (almost 1400 images). Images were downloaded with help of the googliser open source code.

https://www.kaggle.com/datasets/slothkong/10-monkey-species

#Label mapping:
n0, alouattapalliata n1, erythrocebuspatas
n2, cacajaocalvus n3, macacafuscata
n4, cebuellapygmea n5, cebuscapucinus
n6, micoargentatus n7, saimirisciureus
n8, aotusnigriceps n9, trachypithecusjohnii

For more information on the monkey species and number of images per class make sure to check monkey_labels.txt file.
Aim
This dataset is intended as a test case for fine-grain classification tasks, perhaps best used in combination with transfer learning. Hopefully someone can help us expand the number of classes or number of images.

**data and loading in**


In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

from zipfile import ZipFile
file_name='/content/drive/MyDrive/Projects/archive.zip'
with ZipFile(file_name,'r') as Zip:
  Zip.extractall()

Mounted at /content/drive/


Code block [2] did not work as it previously worked with .csv files

I extracted folders to the drive to import directly from it.

In [7]:
train_dir = '/content/drive/MyDrive/Projects/training'
test_dir = '/content/drive/MyDrive/Projects/validation'

In [8]:
train_dir
test_dir

'/content/drive/MyDrive/Projects/validation'

In [9]:
BATCH_SIZE= 16

IMAGE_HEIGHT =224
IMAGE_WIDTH = 224


dataset generator for train

In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale = 1./255,
    rotation_range =90,
    width_shift_range = 0.8,
    height_shift_range = 0.5,
    validation_split =0.3
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [11]:
train_data = train_generator.flow_from_directory(
      train_dir,
      target_size = ( IMAGE_HEIGHT, IMAGE_WIDTH),
      batch_size = BATCH_SIZE,
      subset= 'training'
)

validation_data = train_generator.flow_from_directory(
      train_dir,
      target_size = ( IMAGE_HEIGHT, IMAGE_WIDTH),
      batch_size = BATCH_SIZE,
      subset= 'validation'
)

test_data = train_generator.flow_from_directory(
      train_dir,
      target_size = ( IMAGE_HEIGHT, IMAGE_WIDTH),
      batch_size = BATCH_SIZE
)


Found 769 images belonging to 1 classes.
Found 329 images belonging to 1 classes.
Found 1098 images belonging to 1 classes.


# Model building


In [17]:
pretrained = tf.keras.applications.MobileNet(
    weights = 'imagenet',
    include_top =False,
    pooling ='avg',
    input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
)
pretrained.trainable =False

17235968/17225924 [==============================] - 0s 0us/step
